# setup

In [1]:
import os

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [2]:
import torch
import torch.nn as nn

from models.baseline import ResnetBaseline
from hparams import BATCH_SIZE, NUM_WORKERS

In [3]:
# model_label = 'code151davb'
# from dataloaders.code15 import CODE as DS
# from dataloaders.code15 import CODEsplit as DSsplit
# from runners.solo import Runner
# model = ResnetBaseline(n_classes = 1)
# model = torch.load('output/{}/{}.pt'.format(model_label, model_label))

# output_col = ['1dAVb']

In [4]:
# model_label = 'code151davbflag'
# from dataloaders.code15 import CODE as DS
# from dataloaders.code15 import CODEsplit as DSsplit
# from runners.solo import Runner
# model = ResnetBaseline(n_classes = 1)
# model = torch.load('output/{}/{}.pt'.format(model_label, model_label))

# output_col = ['flag_1dAVb']

In [5]:
# model_label = 'code15flag1davb'
# from runners.multiclass import Runner
# from dataloaders.code15 import CODE as DS
# from dataloaders.code15 import CODEsplit as DSsplit
# model = ResnetBaseline(n_classes = 6)
# model = torch.load('output/{}/{}.pt'.format(model_label, model_label))

# output_col = ['flag_1dAVb', 'RBBB', 'LBBB', 'SB', 'AF', 'ST']

In [6]:
model_label = 'code15'
from runners.multiclass import Runner
from dataloaders.code15 import CODE as DS
from dataloaders.code15 import CODEsplit as DSsplit
model = ResnetBaseline(n_classes = 6)
model = torch.load('output/{}/{}.pt'.format(model_label, model_label))

output_col = ['1dAVb', 'RBBB', 'LBBB', 'SB', 'AF', 'ST']

In [7]:
from dataloaders.codetest import CODEtest
from utils import json_dump

# init

In [8]:
database = DS(metadata_path = '/home/josegfer/datasets/code/output/metadata.csv')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
runner = Runner(device = device, model = model, database = database, split = DSsplit, model_label = model_label, output_col = output_col)

# run

In [10]:
tst_ds = CODEtest()
val_dl = torch.utils.data.DataLoader(runner.val_ds, batch_size = BATCH_SIZE, 
                                     shuffle = False, num_workers = NUM_WORKERS)
tst_dl = torch.utils.data.DataLoader(tst_ds, batch_size = BATCH_SIZE, 
                                     shuffle = False, num_workers = NUM_WORKERS)

In [11]:
best_f1s, best_thresholds = runner._synthesis(val_dl, best_thresholds = None)
all_binary_results, all_true_labels, metrics_dict = runner._synthesis(tst_dl, best_thresholds)
json_dump(metrics_dict, model_label, test = True)

100%|██████████| 7/7 [00:00<00:00, 10.07it/s]
